In [1]:
#!pip install keplergl
#!pip install geopandas
#!pip install sodapy

from sodapy import Socrata
import geopandas as gpd
from keplergl import KeplerGl

In [2]:
#insert your own token to avoid throttling
#found at https://data.cityofchicago.org/profile
token = None

In [3]:
def get_rides(start, end, token=None):
    '''
    Creates a request for the SODA API datasets. 
    
    returns: geopandas coordinates for rides
    '''
    #instantiate the Socrata client
    client = Socrata(
        domain = 'data.cityofchicago.org',
        app_token=token, timeout=100000
    )
    
    #set up the SoSQL call
    select = 'pickup_centroid_location AS geometry, COUNT(trip_id) AS rides'
    group = 'geometry' 
    where = f'trip_start_timestamp between "{start}" and "{end}"'
    
    #make the call for the data, specifying the correct dataset
    results = client.get(
        dataset_identifier='m6dm-c72p',
        select=select, group=group, where=where,
        content_type = 'geojson', limit=100_000
    )
    #close our conneciton
    client.close()
    
    #add data to geopandas
    gdf = gpd.GeoDataFrame.from_features(results).dropna()
    gdf.rides = gdf.rides.astype('int32')
    
    return gdf

In [4]:
# creating the date range we'll use for this notebook
start = '2019-01-01'
end = '2020-01-31'

In [5]:
%%time
#calling our data gathering funciton
gdf_point = get_rides(start, end, token)

CPU times: user 77.1 ms, sys: 48.5 ms, total: 126 ms
Wall time: 8min 1s


In [7]:
map_1 = KeplerGl(height=900)
map_1.add_data(gdf_point, 'rides')
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'rides': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…